# Creating a Story Map Using Leaflet and CSVs

---

**Objectives:**

By the end of this exericse, you should be able to:

* Create an HTML page
* Host an interactive map (Leaflet) on GitHub Pages
* Create a storymap using Python, Leaflet, and CSV files

---

In this exercise, you will learn how to present spatial data in an interactive narrative similar to [ESRI StoryMaps](https://storymaps-classic.arcgis.com/en/gallery/#s=0) using open source products. You will learn how to host a live version of an HTML webpage on GitHub Pages.

<!-- If you double-click on the `raleigh.html` file in your folder, a map will open in your web browser. Next, let's open the `raleigh.html` file in **VS Code**. -->

This code is adapted from the `index.html` file from [this GitHub repository](https://github.com/HandsOnDataViz/leaflet-map-simple).

This exercise is an adaptation of the ["Leaflet Storymaps with Google Sheets" tutorial](https://handsondataviz.org/leaflet-storymaps-with-google-sheets.html) from the *Hands-On Data Visualization* book by Jack Dougherty & Ilya Ilyankou. The open-acess web edition of the book can be found [here](https://handsondataviz.org/).

Dougherty and Ilyankou developed Leaflet Storymaps with Google Sheets to create an open-source, easily migratable, story map tool. In this exercise, we are adapting their work with Google Sheets into using Python to create and edit CSVs to create the open-source storymap.

### Host a Live Leaflet Map on GitHub Pages

In order to host your storymap on the internet, you need to create a GitHub repository for this project. The best way to do this is to **fork** the leaflet-storymap-nr491 repo from [this repo](https://github.com/mdgaines/leaflet-storymap-nr491) on my (Mollie Gaines') GitHub.

Go to Settings > Pages and set Branch to `main` and save.

Edit your README.md to have the link to your new repo.

### Leaflet Storymap with CSVs

We are now going to start working on our Leaflet Storymap by building several `pandas dataframes` which we can save as a CSVs.

Our storymap will have a scrolling narrative that follows points along our map. You will learn how to add images, audio and video files, and different map backgrounds. 

We will build an in-class storymap to looking at different colleges and universities in Wake County.

In [149]:
import pandas as pd
import geopandas as gpd
import os
import folium
import csv

First, we will create an `Options.csv` file and save it in our `csv/` directory. We will create this CSV to look like the **Options** tab in the Google Sheet from the HODV exercise.

![GS_Options](./imgs/gs_options.PNG)

In [150]:
# Create lists for each row we are writing in our csv
header = ['Setting', 'Customize', 'Hints']
info = ['Storymap Info', '', \
    'For help, see tutorial in [HandsOnDataViz.org](https://handsondataviz.org/leaflet-storymaps-with-google-sheets.html)']
title = ['Storymap Title', 'NR491 Cost of Sea Level Rise', '']
subtitle = ['Storymap Subtitle', \
    "Measuring the Economic Impact of Sea Level Rise on Local North Carolina Community<br><small>Scroll down<i class='fafa-chevron-down'></i></small>",\
    "Add your subtitle, or delete that portion"]
logo = ['Storymap Logo', 'media/ncsu_logo.jpg', 'Path to a logo image']
google_analysis = ['Google Analytics Tracking ID', '', 'Sample format: UA-5488840-29']

map_settings = ['Map Settings', '', '']
basemap_tiles = ['Basemap Tiles', 'CartoDB.Positron',\
    '[Drop-down menu for background basemap tiles](https://leaflet-extras.github.io/leaflet-providers/preview/)']
zoom = ['Zoom Controls', 'bottomright', '']
bg_color = ['Narrative Background Color', '', '']
txt_color = ['Narrative Text Color', '', '']
lnk_color = ['Narrative Link Color', '', '']
ch_color = ['Active Chapter Background Color','','']
media_height = ['Media Container Height','300',\
    'Maximum height of the image, in pixels. 200 is default. The image will be fit into the container with its proportions kept (it won\'t be skewed).']
pixels_after_char = ['Pixels After Final Chapter','600','In pixels, at least 100']
lightbox_imgs = ['Enable Lightbox for Images','yes','yes or no']

creds = ['Credits','','']
auth_name = ['Author Name','Collin Yates','Appears in map credits as "View data by..." (or leave blank)']
auth_email = ['Author Email or Website','','Create link in Author Name by inserting your email or web address (or leave blank)']
auth_GH = ['Author GitHub Repo Link','https://github.com/handsondataviz/leaflet-storymaps-with-google-sheets',\
    'Insert your code repo URL to appear as link in "View code..." (or leave blank)']
code_cred = ['Code Credit','<a href="https://handsondataviz.org">HandsOnDataViz</a>',\
    'Appears in credits as "...code by..." (or leave blank)']

**NOTE:** you can have several different basemap tiles that will appear in the background. 
The options we will look at are:

![Basemaps](./imgs/gs_basemap_tiles.PNG)

We can also choose where on our map we put our zoom controls. The options are:

![zoom](./imgs/gs_zoom.PNG)

Now, we'll write our first CSV.

In [151]:
# Check that the csv directory exists
if not os.path.exists('./csv'):
    os.mkdir('./csv')

# Create a new Options.csv

# opens a csv file in write mode
# the wuth statement makes sure the file closes properly when we're done
with open('./csv/Options_Template.csv', 'w', newline='') as options:
    # create the csv writer
    writer = csv.writer(options)

    # write a row to the csv file
    writer.writerow(header)

    writer.writerow(info)
    writer.writerow(title)
    writer.writerow(subtitle)
    writer.writerow(logo)
    writer.writerow(google_analysis)

    writer.writerow(map_settings)
    writer.writerow(basemap_tiles)
    writer.writerow(zoom)
    writer.writerow(bg_color)
    writer.writerow(txt_color)
    writer.writerow(lnk_color)
    writer.writerow(ch_color)
    writer.writerow(media_height)
    writer.writerow(pixels_after_char)
    writer.writerow(lightbox_imgs)

    writer.writerow(creds)
    writer.writerow(auth_name)
    writer.writerow(auth_email)
    writer.writerow(auth_GH)
    writer.writerow(code_cred)


Now that we've saved our CSV, we will read it in as a Pandas DataFrame and make some edits so it is personalized for us.

**Note:** We could have written our original CSV with all the information we wanted, however now we can practice making edits.

In [152]:
# read in the options csv as a data frame
options_df = pd.read_csv('./csv/Options_Template.csv', index_col=0)
options_df

,Customize,Hints
Setting,,
Storymap Info,NaN,"For help, see tutorial in [HandsOnDataViz.org]..."
Storymap Title,NR491 Cost of Sea Level Rise,NaN
Storymap Subtitle,Measuring the Economic Impact of Sea Level Ris...,"Add your subtitle, or delete that portion"
Storymap Logo,media/ncsu_logo.jpg,Path to a logo image
Google Analytics Tracking ID,NaN,Sample format: UA-5488840-29
Map Settings,NaN,NaN
Basemap Tiles,CartoDB.Positron,[Drop-down menu for background basemap tiles](...
Zoom Controls,bottomright,NaN
Narrative Background Color,NaN,NaN


In [153]:
# we will index to the specific entry we want to edit using df[col][row] indexing.
# we read in the df with the first column as our index to make this easier for ourselves

# here, we are changing the background color of our narative bar
options_df['Customize']['Narrative Background Color'] = 'lightgray'

options_df

,Customize,Hints
Setting,,
Storymap Info,NaN,"For help, see tutorial in [HandsOnDataViz.org]..."
Storymap Title,NR491 Cost of Sea Level Rise,NaN
Storymap Subtitle,Measuring the Economic Impact of Sea Level Ris...,"Add your subtitle, or delete that portion"
Storymap Logo,media/ncsu_logo.jpg,Path to a logo image
Google Analytics Tracking ID,NaN,Sample format: UA-5488840-29
Map Settings,NaN,NaN
Basemap Tiles,CartoDB.Positron,[Drop-down menu for background basemap tiles](...
Zoom Controls,bottomright,NaN
Narrative Background Color,lightgray,NaN


In [154]:
# Next, make edits to the Author Name and Author Github Repo Link so it has your information
options_df['Customize']['Author Name'] = 'Collin Yates'
options_df['Customize']['Author GitHub Repo Link'] = 'https://github.com/cayates2/Final-Project-nr491'
options_df['Customize']['Storymap Title'] = 'The Cost of Sea Level Rise'
options_df['Customize']['Storymap Subtitle'] = 'Story map discussing sea level rise and its economic impact on local communities in North Carolina'
# save your options df as Options.csv

options_df.to_csv('./csv/Options.csv')


Now let's checkout what our index.html file looks like so far.

In [155]:
options_df

,Customize,Hints
Setting,,
Storymap Info,NaN,"For help, see tutorial in [HandsOnDataViz.org]..."
Storymap Title,The Cost of Sea Level Rise,NaN
Storymap Subtitle,Story map discussing sea level rise and its ec...,"Add your subtitle, or delete that portion"
Storymap Logo,media/ncsu_logo.jpg,Path to a logo image
Google Analytics Tracking ID,NaN,Sample format: UA-5488840-29
Map Settings,NaN,NaN
Basemap Tiles,CartoDB.Positron,[Drop-down menu for background basemap tiles](...
Zoom Controls,bottomright,NaN
Narrative Background Color,lightgray,NaN


### Chapters

We will add content to our map by creating a `Chapters.csv`.

However, first we will look through our data to get the information we are interested in.

Now, let's read in some of our spatial data.

In [156]:
# Filepaths

beaufort_plots_path = './data/Zonal_Stats_Beaufort/Zonal_Stats_Beaufort.shp'


beaufort_plots = gpd.read_file(beaufort_plots_path)


print(beaufort_plots.crs)


epsg:2264


In [157]:
# Reproject to WGS84
# nc_colleges_wgs84 = nc_colleges.to_crs(epsg=4326)
# nc_amtrak_stations_wgs84 = nc_amtrak_stations.to_crs(epsg=4326)

beaufort_plots_wgs84 = beaufort_plots.to_crs(epsg=4326)

# Create a Geo-id which is needed by the Folium (it needs to have a unique identifier for each row)
# nc_colleges_wgs84['geoid'] = nc_colleges_wgs84.index.astype(str)
# nc_amtrak_stations_wgs84['geoid'] = nc_amtrak_stations_wgs84.index.astype(str)

beaufort_plots_wgs84['geoid'] = beaufort_plots_wgs84.index.astype(str)

We can use the `folium` package to see what our Leaflet map will look like.

In [182]:
# Create a Map instance centered on Raleigh
m = folium.Map(location=[35.4567, -48.2239], zoom_start=5, control_scale=True)
m

In [160]:
# Map_Locations = pd.read_csv('./csv/Map_Locations.csv')

In [161]:
# Show the colleges on the map

# Create college points on top of the map
#for idx, row in Map_Locations.iterrows():
    # Get lat and lon of points
   # lon = row['x']
   # lat = row['y']

    # Get college information
   # college_name = row['NAME']

    #Add markers to the map
   # folium.Marker([lat,lon], popup=college_name).add_to(m)

m

In [167]:
chapters_df = pd.read_csv('./csv/Chapters_Project.csv')
chapters_df

,Unnamed: 0,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties
0,0,Overview of Atlantic,media/gulf_stream.png,Source: USGS,https://www.usgs.gov/media/images/map-north-at...,This is a test.,3.0,Hidden,NaN,North Atlantic,35.503600,-69.423450,NaN,NaN,NaN,NaN
1,1,North Carolina,media/peace.jpg,Image Source: Niche,https://www.niche.com/colleges/william-peace-u...,William Peace University is a private college ...,16.0,Plain,blue,15 E PEACE ST,35.788240,-78.637376,NaN,NaN,NaN,NaN
2,2,Beaufort,NaN,NaN,NaN,NaN,16.0,Plain,blue,118 E SOUTH ST,35.771817,-78.637986,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
# chapter_df = pd.read_csv('./csv/Chapters.csv')
# chapter_df

chapter_df = pd.read_csv('./csv/Chapters_Project.csv')

chapter_df

,Unnamed: 0,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties
0,0,Overview of Atlantic,media/gulf_stream.png,Source: USGS,https://www.usgs.gov/media/images/map-north-at...,This is a test.,3.0,Hidden,NaN,North Atlantic,35.503600,-69.423450,NaN,NaN,NaN,NaN
1,1,North Carolina,media/peace.jpg,Image Source: Niche,https://www.niche.com/colleges/william-peace-u...,William Peace University is a private college ...,16.0,Plain,blue,15 E PEACE ST,35.788240,-78.637376,NaN,NaN,NaN,NaN
2,2,Beaufort,NaN,NaN,NaN,NaN,16.0,Plain,blue,118 E SOUTH ST,35.771817,-78.637986,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Our first chapter will give a brief overview of Raleigh. We will include a fun skyline image with a link to where we got the image (the Raleigh government webpage). Our map will show a historical, georeferenced map of the Raleigh City Plan from 1834 along with blue markers for each of the 8 schools we will look at for chapters.

To include images in our narative, we need to have them downloaded to our `media` directory.

In [169]:
chapter_df.columns

Index(['Unnamed: 0', 'Chapter', 'Media Link', 'Media Credit',
       'Media Credit Link', 'Description', 'Zoom', 'Marker', 'Marker Color',
       'Location', 'Latitude', 'Longitude', 'Overlay', 'Overlay Transparency',
       'GeoJSON Overlay', 'GeoJSON Feature Properties'],
      dtype='object')

In [171]:
chapter_df

,Unnamed: 0,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties
0,0,Overview of Atlantic,media/gulf_stream.png,Source: USGS,https://www.usgs.gov/media/images/map-north-at...,This is a test.,3.00,Hidden,NaN,North Atlantic,35.503600,-69.423450,NaN,NaN,NaN,NaN
1,1,North Carolina,media/peace.jpg,Image Source: Niche,https://www.niche.com/colleges/william-peace-u...,William Peace University is a private college ...,16.00,Plain,blue,15 E PEACE ST,35.788240,-78.637376,NaN,NaN,NaN,NaN
2,2,Beaufort,NaN,NaN,NaN,NaN,16.00,Plain,blue,118 E SOUTH ST,35.771817,-78.637986,NaN,NaN,NaN,NaN
3,3,NaN,media/visit_raleigh.jpg,Source: Raleigh Government,https://raleighnc.gov/,"TESTING Before European colonization, the Rale...",15.75,Hidden,,"Raleigh, NC",35.779600,-78.638200,https://mapwarper.net/maps/tile/51816/{z}/{x}/...,0.9,NaN,NaN


In [172]:
# nc_coll_raleigh = nc_coll_raleigh.reset_index()
# nc_coll_raleigh.head()

In [173]:
# for i in nc_coll_raleigh.index:
    # print(nc_coll_raleigh.iloc[i])
   # school_name = nc_coll_raleigh.loc[[i], ['NAME']].values[0][0].title()
   # lat = nc_coll_raleigh.loc[[i], ['Y']].values[0][0]
   # lon = nc_coll_raleigh.loc[[i], ['X']].values[0][0]
   # address = nc_coll_raleigh.loc[[i], ['ADDRESS']].values[0][0]

  #  chapter_df.loc[chapter_df['Chapter'] == school_name, 'Latitude'] = lat
   # chapter_df.loc[chapter_df['Chapter'] == school_name, 'Longitude'] = lon
  #  chapter_df.loc[chapter_df['Chapter'] == school_name, 'Location'] = address

  #  chapter_df.loc[chapter_df['Chapter'] == school_name, 'Zoom'] = 16
   # chapter_df.loc[chapter_df['Chapter'] == school_name, 'Marker'] = 'Plain'
   # chapter_df.loc[chapter_df['Chapter'] == school_name, 'Marker Color'] = 'blue'

chapter_df


,Unnamed: 0,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties
0,0,Overview of Atlantic,media/gulf_stream.png,Source: USGS,https://www.usgs.gov/media/images/map-north-at...,This is a test.,3.00,Hidden,NaN,North Atlantic,35.503600,-69.423450,NaN,NaN,NaN,NaN
1,1,North Carolina,media/peace.jpg,Image Source: Niche,https://www.niche.com/colleges/william-peace-u...,William Peace University is a private college ...,16.00,Plain,blue,15 E PEACE ST,35.788240,-78.637376,NaN,NaN,NaN,NaN
2,2,Beaufort,NaN,NaN,NaN,NaN,16.00,Plain,blue,118 E SOUTH ST,35.771817,-78.637986,NaN,NaN,NaN,NaN
3,3,NaN,media/visit_raleigh.jpg,Source: Raleigh Government,https://raleighnc.gov/,"TESTING Before European colonization, the Rale...",15.75,Hidden,,"Raleigh, NC",35.779600,-78.638200,https://mapwarper.net/maps/tile/51816/{z}/{x}/...,0.9,NaN,NaN


Now, let's save this to a CSV and see how this changes our storymap.

In [174]:
chapter_df.to_csv('./csv/Chapters.csv')

We have a map! We also have some images, links, and narrative text appearing.

Now, let's add a new image and image source information for Peace College.

In [175]:
# Add an image for our Peace College Chapter
# chapter_df.loc[1, "Media Link"] = "media/peace.jpg"

# add source name
# chapter_df.loc[1, "Media Credit"] = "Image Source: Niche"

# add the source link
# chapter_df.loc[1, "Media Credit Link"] = "https://www.niche.com/colleges/william-peace-university/"

In [186]:
chapters_df.columns

Index(['Unnamed: 0', 'Chapter', 'Media Link', 'Media Credit',
       'Media Credit Link', 'Description', 'Zoom', 'Marker', 'Marker Color',
       'Location', 'Latitude', 'Longitude', 'Overlay', 'Overlay Transparency',
       'GeoJSON Overlay', 'GeoJSON Feature Properties'],
      dtype='object')

In [189]:
chapter_df.loc[1, "Chapter"] = "Effects on North Carolina"
chapter_df.loc[1, "Media Link"] = "media/wilm_change.png"
chapter_df.loc[1, "Media Credit"] = "sealevelrise.org"
chapter_df.loc[1, "Media Credit Link"] = "https://sealevelrise.org/states/north-carolina/#:~:text=And%20It's%20Costing%20Over%20%242%20Billion&text=This%20increase%20is%20mostly%20due,and%20vulnerability%20to%20coastal%20storms."
chapter_df.loc[1, "Description"] = "SEA LEVEL MEASUREMENT FROM WILMINGTON AREA TIDE GAUGE SINCE 1950"
chapter_df.loc[1, "Zoom"] = "10"
chapter_df.loc[1, "Marker"] = "plain"
chapter_df.loc[1, "Marker Color"] = "blue"
chapter_df.loc[1, "Location"] = "Wilmington, NC"
chapter_df.loc[1, "Latitude"] = "34.20860292498924"
chapter_df.loc[1, "Longitude"] = "77.89353114975715"

In [ ]:
chapter_df.loc[2, "Chapter"] = ""
chapter_df.loc[2, "Media Link"] = ""
chapter_df.loc[2, "Media Credit"] = ""
chapter_df.loc[2, "Media Credit Link"] = ""
chapter_df.loc[2, "Description"] = ""
chapter_df.loc[2, "Zoom"] = ""
chapter_df.loc[2, "Marker"] = ""
chapter_df.loc[2, "Marker Color"] = ""
chapter_df.loc[2, "Location"] = ""
chapter_df.loc[2, "Latitude"] = ""
chapter_df.loc[2, "Longitude"] = ""

In [190]:
# Check our work
chapter_df.head()

,Unnamed: 0,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties
0,0,Overview of Atlantic,media/gulf_stream.png,Source: USGS,https://www.usgs.gov/media/images/map-north-at...,This is a test.,3.0,Hidden,NaN,North Atlantic,35.5036,-69.42345,NaN,NaN,NaN,NaN
1,1,Effects on North Carolina,media/wilm_change.png,sealevelrise.org,https://sealevelrise.org/states/north-carolina...,SEA LEVEL MEASUREMENT FROM WILMINGTON AREA TID...,10,plain,blue,"Wilmington, NC",34.20860292498924,77.89353114975715,NaN,NaN,NaN,NaN
2,2,Effects on North Carolina,media/wilm_change.png,sealevelrise.org,https://sealevelrise.org/states/north-carolina...,SEA LEVEL MEASUREMENT FROM WILMINGTON AREA TID...,10,plain,blue,"Wilmington, NC",34.20860292498924,77.89353114975715,NaN,NaN,NaN,NaN
3,3,NaN,media/visit_raleigh.jpg,Source: Raleigh Government,https://raleighnc.gov/,"TESTING Before European colonization, the Rale...",15.75,Hidden,,"Raleigh, NC",35.7796,-78.6382,https://mapwarper.net/maps/tile/51816/{z}/{x}/...,0.9,NaN,NaN


In [177]:
# Add to the Description of Peace College including when it was founded and an interesting fact about it
# chapter_df.loc[1, "Description"] = "William Peace University is a private college in Raleigh, North Carolina. Formerly affiliated with the Presbyterian Church, it offers undergraduate degrees in more than 30 majors and the School of Professional Studies offers accelerated bachelor degrees that are online or hybrid for working adults."

# chapter_df.head()

Next, we'll look at adding a video to our narrative bar.

In order to add a YouTube video to our storymap, we need to first find a YouTube video we want to add. For this example, let's follow [this link](https://www.youtube.com/watch?v=aZQFQnbs_Ho) to a fan-made hype video of the NC State Football Team.

Next, click the `Share` button at the bottom right of the video and then click `Embed`. Next, **only** copy the link that appears after `src="`. We will use that in our next code chunk.

<!-- Link that should be copied: https://www.youtube.com/embed/aZQFQnbs_Ho" -->

In [178]:
# Add our copied embed youtube link to the "Media Link" column for NC State
# chapter_df.loc[4, "Media Link"] = "https://youtu.be/aZQFQnbs_Ho"

# Save to Chapters.csv 

# chapter_df.to_csv('./csv/Chapters.csv')

Let's look at our `index.html` again. What has changed? Does it look the way you expect?

### Overlay GeoJSON

Next, we'll add a GeoJSON file to our map. Leaflet uses GeoJSONs instead of Shapefiles. If you mostly use Shapefiles, just open them into QGIS and save them as GeoJSONs to get them ready for the storymap.

All of our GeoJSONs need be saved in the `geojson` folder.

**Important:** Your GeoJSON files **MUST** be in a recognized projection and have a `"crs"` attribute. *It will be easiest for you if they are all in the same CRS.*

Let's add our GeoJSON of NC State's campuses.

In [179]:
#chapter_df.loc[4, "GeoJSON Overlay"] = "geojson/NCSU_campuses.geojson"

# We can also set different parameters for the GeoJSON
# These parameters are: fillColor, weight, opacity, color, and fillOpacity
#chapter_df.loc[4, "GeoJSON Feature Properties"] = "fillColor:red;color:black"

# save the Chapters.csv and look at the index.html file
#chapter_df.to_csv('./csv/Chapters.csv')

To change a specific campus color, we will need to edit the GeoJSON itself.

*NOTE: I am working on integrating another set of Leaflet code so that you will not have to manually edit the GeoJSON files, but it's not ready yet.*

To edit the GeoJSON file, open it with a text-editor (like Notepad) or in VS Code. Use `ctrl+f` to find `"properties"`. GeoJSON files are essentially full of dictionaries. After the `"Precinct_N"` key:value pair, add `"fillColor": "blue",` and save your changes.

Refresh your `index.html`. Do you see any changes? (Note: it can take a while for changes to be fully processed. Make sure everything is saved.)

In [180]:
# Add Wake county census tracts to your map for a chapter of your choice


# Try to change the census tract for the school in your chapter to a different color

Now, let's add another historic map of Raleigh. Go to https://mapwarper.net/ and put "raleigh" in the search bar. Select "Rectified maps only" and search. 

Look through the maps and find one you want to use. Click on the "Export" tab at the top and copy the link from the "Tiles (Google/OSM scheme):" option.

<!-- https://mapwarper.net/maps/tile/44859/{z}/{x}/{y}.png -->

In [181]:
# Add this link to the Overlay column to a chapter of your choice
# remember, if the historical map does not overlap with the school for that chapter, 
# we will need to manually zoom out to see it or change our zoom settings



Try adding new images, text, vidoes, and GeoJSON files to different chapters. 
Remember to add the data we are adding to their respective folders.